In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import numpy as np
from utils.logger import PrettyLogger
from utils.io_func import (
    load_from_tiff, save_to_tiff, load_from_npy, save_to_npy
)
from config import START_V_I, START_H_I, SIDE_LEN

In [ ]:
logger = PrettyLogger()

In [ ]:
SITE_YEAR_TUPLE = (("Site_A", "2015"),)
DATA_DIR_TEMPLATE = "../data/{site}/CDL/"
CROPPED_FILEPATH_TEMPLATE = "./out/{site}/CDL/cropped/CDL-{year}.tif"
TRANSCODED_FILEPATH_TEMPLATE = "./out/{site}/CDL/transcoded/CDL-{year}.tif"
INTERPOLATED_FILEPATH_TEMPLATE = (
    "./out/{site}/ARD/cropped_interpolated/{year}/filter_band.npy"
)
FINAL_OUT_FILEPATH_TEMPLATE = "./out/{site}/y-{year}.npy"

# Cropping and transcoding

In [ ]:
def transcode(p):
    # corn:1 -> 1, soybean:5 -> 2, other:other -> 0
    return 1 if p == 1 else 2 if p == 5 else 0


def preprocess_cdl(
    data_path, cropped_filepath, transcoded_filepath,
    interpolated_filepath, final_out_filepath
):
    # input
    raw_img = load_from_tiff(data_path)

    # crop cdl
    cropped_img = raw_img[
        START_V_I:START_V_I + SIDE_LEN, START_H_I:START_H_I+SIDE_LEN
    ]
    save_to_tiff(cropped_img, cropped_filepath)

    # transcoding
    transcoded_img = np.vectorize(transcode)(cropped_img)
    save_to_tiff(transcoded_img, transcoded_filepath)

    # remove invalid pixels
    filter_band = load_from_npy(interpolated_filepath)
    y = transcoded_img[filter_band]

    # output preprocessed data
    save_to_npy(y, final_out_filepath)

In [ ]:
for site, year in SITE_YEAR_TUPLE:
    data_dir = DATA_DIR_TEMPLATE.format(site=site)
    filename = list(filter(lambda x: x[4:8] == year, os.listdir(data_dir)))[0]
    data_path = os.path.join(data_dir, filename)
    logger.info("Processing:" site, filename)
    preprocess_cdl(
        data_path=data_path,
        cropped_filepath=CROPPED_FILEPATH_TEMPLATE.format(
            site=site, year=year
        ),
        transcoded_filepath=TRANSCODED_FILEPATH_TEMPLATE.format(
            site=site, year=year
        ),
        interpolated_filepath=INTERPOLATED_FILEPATH_TEMPLATE.format(
            site=site, year=year
        ),
        final_out_filepath=FINAL_OUT_FILEPATH_TEMPLATE.format(
            site=site, year=year
        )
    )